In [14]:
import pandas as pd
dataset=pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [15]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [16]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [17]:
indep=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
dep=dataset[['charges']]

In [18]:
from sklearn.preprocessing import StandardScaler
scx = StandardScaler()
indep=scx.fit_transform(indep)
indep

array([[-1.43876426, -0.45332   , -0.90861367, -1.0105187 ,  1.97058663],
       [-1.50996545,  0.5096211 , -0.07876719,  0.98959079, -0.5074631 ],
       [-0.79795355,  0.38330685,  1.58092576,  0.98959079, -0.5074631 ],
       ...,
       [-1.50996545,  1.0148781 , -0.90861367, -1.0105187 , -0.5074631 ],
       [-1.29636188, -0.79781341, -0.90861367, -1.0105187 , -0.5074631 ],
       [ 1.55168573, -0.26138796, -0.90861367, -1.0105187 ,  1.97058663]])

In [19]:
scy = StandardScaler()
dep= scy.fit_transform(dep)
dep

array([[ 0.2985838 ],
       [-0.95368917],
       [-0.72867467],
       ...,
       [-0.96159623],
       [-0.93036151],
       [ 1.31105347]])

In [20]:
!pip install scikit-learn

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
param_grid={
    'criterion':['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
    'n_estimators':[50,100],
    'max_features':['sqrt','log2',None]
}
grid=GridSearchCV(RandomForestRegressor(), param_grid, refit=True, verbose=3, n_jobs=-1)
grid.fit(indep,dep)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
30 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py", line 409, in fit
    raise ValueError(
ValueError: Some value(s) of y are negative wh

GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error',
                                       'friedman_mse', 'poisson'],
                         'max_features': ['sqrt', 'log2', None],
                         'n_estimators': [50, 100]},
             verbose=3)

In [22]:
grid.best_params_

{'criterion': 'friedman_mse', 'max_features': 'sqrt', 'n_estimators': 100}

In [23]:
y_pred=grid.predict(indep)
y_pred

array([ 0.31651716, -0.94322284, -0.69407617, ..., -0.91954247,
       -0.92035056,  1.26371603])

In [24]:
from sklearn.metrics import r2_score
r2=r2_score(y_pred,dep)
print(f"R2 score = {r2:.2f}")

R2 score = 0.97


In [25]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.320858,0.019108,0.010088,0.000028,squared_error,sqrt,50,"{'criterion': 'squared_error', 'max_features':...",0.859587,0.771930,0.858785,0.830100,0.835618,0.831204,0.031939,10
1,0.551637,0.004182,0.018253,0.004099,squared_error,sqrt,100,"{'criterion': 'squared_error', 'max_features':...",0.859096,0.777100,0.862723,0.827111,0.844221,0.834050,0.031127,3
2,0.274065,0.005177,0.010093,0.006396,squared_error,log2,50,"{'criterion': 'squared_error', 'max_features':...",0.857180,0.774724,0.850091,0.828457,0.839375,0.829965,0.029285,12
3,0.544262,0.005315,0.018423,0.003801,squared_error,log2,100,"{'criterion': 'squared_error', 'max_features':...",0.858922,0.778891,0.860804,0.829149,0.837674,0.833088,0.029697,5
4,0.361150,0.006998,0.012088,0.004043,squared_error,None,50,"{'criterion': 'squared_error', 'max_features':...",0.846739,0.766308,0.855216,0.818715,0.839205,0.825236,0.031848,16
5,0.729275,0.007865,0.016128,0.004953,squared_error,None,100,"{'criterion': 'squared_error', 'max_features':...",0.845089,0.776266,0.859011,0.820350,0.846336,0.829411,0.029376,13
6,0.888194,0.012376,0.012087,0.004039,absolute_error,sqrt,50,"{'criterion': 'absolute_error', 'max_features'...",0.854207,0.777373,0.857253,0.826482,0.842066,0.831476,0.029139,9
7,1.785149,0.065698,0.020540,0.008835,absolute_error,sqrt,100,"{'criterion': 'absolute_error', 'max_features'...",0.856487,0.778503,0.859908,0.826778,0.839422,0.832220,0.029402,7
8,0.949956,0.054522,0.002022,0.004045,absolute_error,log2,50,"{'criterion': 'absolute_error', 'max_features'...",0.852495,0.787497,0.851117,0.820510,0.842148,0.830753,0.024467,11
9,1.747819,0.021738,0.022440,0.005589,absolute_error,log2,100,"{'criterion': 'absolute_error', 'max_features'...",0.857317,0.781872,0.855631,0.822396,0.843641,0.832172,0.028071,8


In [26]:
# age=int(input("Age:"))
# bmi=float(input("BMI:"))
# children=int(input("Children:"))
# sex_male=int(input("Sex_Male: 0/1"))
# smoker_yes=int(input("Smoker_yes 0 / 1: "))

# Future_predictions=grid.predict([[age,bmi,children,sex_male,smoker_yes]])
# print("Future predictions={}".format(Future_predictions))